In [ ]:
import torch

# Load HAR data
train_data = torch.load('data/HAR/train.pt')
X_train = train_data['samples']  # [N, 9, 128]
Y_train = train_data['labels']   # [N]

N, C, L = X_train.shape  # e.g., N=7352, C=9, L=128
X_univariate = X_train.reshape(N * C, L).float()  # [N×9, 128] = [66168, 128]

In [ ]:
X_univariate.shape, Y_train.shape

In [ ]:
from hierarchical_event_labeling import CompleteHierarchicalEventDataset

event_dataset = CompleteHierarchicalEventDataset(
    X_univariate,  # [66168, 128] - univariate
    use_spectral=True,
    use_wavelets=True,
    use_wavelet_peaks=True,
    use_changepoint=True,
    use_chaotic=False,
    verbose=True
)
# Creates 66168 univariate annotations (one per channel)

In [ ]:
from data_loader import create_shuffled_dataloader

# Step 4: Create shuffled dataloader
train_loader = create_shuffled_dataloader(
    original_data=X_train,
    hierarchical_dataset=event_dataset,
    batch_size=10,
    shuffle_samples=True,
    verbose=True
)


In [ ]:
from hierarchical_event_labeling import VOCAB

# Get one sample
sample = train_loader.dataset[1]
channel_0_intervals = sample['intervals'][0]  # First channel


# Print with label names
for start, end, label_id in channel_0_intervals:
    label_name = VOCAB.id_to_label(label_id)
    duration = end - start + 1
    print(f"[{start:3d}, {end:3d}] {label_name:<30} (duration: {duration:3d})")


In [ ]:
# Step 5: Training (intervals shuffled each epoch!)
for epoch in range(2):
    print(f"\n=== Epoch {epoch} ===")
    
    for batch in train_loader:
        timeseries = batch['timeseries']        # [32, 9, 128] - FIXED
        target_seq = batch['target_sequence']   # [32, max_len] - SHUFFLED
        print("Timeseries shape:", timeseries)
        print(target_seq)

In [ ]:
target_seq.cuda().device